<a href="https://colab.research.google.com/github/st-tran/CSC413-Project/blob/main/CSC413_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSC413 Project: GeoGuessr!


## A bunch of imports

In [96]:
from typing import *
from collections import defaultdict
from PIL import Image
from IPython.display import display
from glob import glob
import re
import os
from random import shuffle
import time

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
from torchvision import transforms

## Visualizing stuff

In [42]:
def visualize_example(tup):
  dst = Image.new("RGB", (1024, 256))
  dst.paste(tup[0][0], (0, 0))
  dst.paste(tup[0][1], (256, 0))
  dst.paste(tup[0][2], (512, 0))
  dst.paste(tup[0][3], (768, 0))
  display(dst)

## Load in the data

In [ ]:
# 1k Canada dataset
!gdown --id 1QSVxo_GpPQ5gHVh6hkQCEktt6jGX3AoO
!mkdir /content/training
!tar -xzvf 1kimages090180270.tar.gz -C /content/training
!for d in /content/training/images*; do mv "${d}" "/content/training/`basename "${d}" | sed -e 's/images//g' | sed -e 's/\s//g'`"; done
!rm -rf /content/1kimages090180270.tar.gz

In [8]:
data = defaultdict(list)

def get_image_ids(province: str) -> List[str]:
  images = glob(f"/content/training/{province}/*.jpg")
  ids_no_angle = [re.sub('_[\\d]*.jpg', '', f) for f in images]
  return ids_no_angle 

def stitch_images_and_save(image_base_path: str) -> None:
  img = Image.new("RGB", (512, 512))
  img_0 = Image.open(f"{image_base_path}_0.jpg")
  img_90 = Image.open(f"{image_base_path}_90.jpg")
  img_180 = Image.open(f"{image_base_path}_180.jpg")
  img_270 = Image.open(f"{image_base_path}_270.jpg")
  img.paste(img_0, (0, 0))
  img.paste(img_90, (256, 0))
  img.paste(img_180, (0, 256))
  img.paste(img_270, (256, 256))

  new_name = f"{image_base_path}_stitched.jpg"
  img.save(new_name)
  data[image_base_path.split("/")[-2]].append(new_name)

## A custom dataset

In [101]:
class GeoDataset(Dataset):
  """
  Holds a bunch of (N, E, S, W) quadruplets of examples along with the corresponding province labels.

  # Todo: maybe add transform?
  """
  def __init__(self, index_file, transform=None, shuf=False):
    self.index_file = index_file
    self.transform = transform

    self.labels = index_files.keys()

    # Data has (N, E, S, W, province)
    self.data = []
    for prov, tups in index_file.items():
      for tup in tups:
        self.data.append((*tup, prov))
    if shuf:
      shuffle(self.data)

  def __len__(self) -> int:
    return len(self.data)

  def __getitem__(self, idx):
    """
    Returns: 
      (N, E, S, W) images, province
    tuple
    """
    tup = self.data[idx]
    return tuple(
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
          (ToTensor()(Image.open(t))) for t in tup[:-1]
        ), tup[-1]


In [85]:
index_files = defaultdict(list)

for province in glob("/content/training/*"):
  name = os.path.basename(province)
  for image in get_image_ids(name):
    # stitch_images_and_save(image)
    index_files[name].append(tuple(f"{image}_{angle}.jpg" for angle in ("0", "90", "180", "270")))

In [102]:
dataset = GeoDataset(index_files, shuf=True)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [105]:
images, labels = next(iter(loader))

## Code to actually do the training logic

In [ ]:
class AttrDict(dict):
  def __init__(self, *args, **kwargs):
    super(AttrDict, self).__init__(*args, **kwargs)
    self.__dict__ = self


def compute_loss(criterion, outputs, labels, batch_size, num_colours):
  """ Copied from A2 """
  loss_out = outputs.transpose(1, 3).contiguous().view([-1, num_colors])
  loss_lab = labels.transpose(1, 3).contiguous().view([-1])
  return criterion(loss_out, loss_lab)

def train(args, cnn=None):
  save_dir = f"/outputs/{args.experiment_name}"
  num_labels = len(dataset.labels)
  num_in_channels = 3

  if cnn is None:
    Net = globals()[args.model]
    cnn = Net(args.kernel, args.num_filters, num_colours, num_in_channels)

  # Loss function
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(cnn.parameters(), lr=args.learn_rate)

  print("Beginning training...")
  if args.gpu:
    cnn.cuda()
  start = time.time()

  train_losses = []
  valid_losses = []
  valid_accs = []
  for epoch in range(args.epochs):
    # Train the model
    cnn.train()
    losses = []
    for i, (images, labels) in enumerate(dataset):
      # Forward + Backward + Optimize
      optimizer.zero_grad()
      outputs = cnn(images)

      loss = compute_loss(
          criterion, outputs, labels, batch_size=args.batch_size, num_colours=num_colours
      )

      loss.backward()
      optimizer.step()
      losses.append(loss.data.item())

      # plot training images
      avg_loss = np.mean(losses)
      train_losses.append(avg_loss)
      time_elapsed = time.time() - start
      print(
          "Epoch [%d/%d], Loss: %.4f, Time (s): %d"
          % (epoch + 1, args.epochs, avg_loss, time_elapsed)
      )

      # Evaluate the model
      cnn.eval()  # Change model to 'eval' mode (BN uses moving mean/var).
      val_loss, val_acc = run_validation_step(
          cnn,
          criterion,
          test_grey,
          test_rgb_cat,
          args.batch_size,
          colours,
          save_dir + "/test_%d.png" % epoch,
          args.visualize,
          args.downsize_input,
      )

      time_elapsed = time.time() - start
      valid_losses.append(val_loss)
      valid_accs.append(val_acc)
      print(
          "Epoch [%d/%d], Val Loss: %.4f, Val Acc: %.1f%%, Time(s): %.2f"
          % (epoch + 1, args.epochs, val_loss, val_acc, time_elapsed)
      )

  # Plot training curve
  plt.figure()
  plt.plot(train_losses, "ro-", label="Train")
  plt.plot(valid_losses, "go-", label="Validation")
  plt.legend()
  plt.title("Loss")
  plt.xlabel("Epochs")
  plt.savefig(save_dir + "/training_curve.png")

  if args.checkpoint:
      print("Saving model...")
      torch.save(cnn.state_dict(), args.checkpoint)

  return cnn


## 18-Layer Resnet